In [1]:
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# 실제 컬럼명으로 수정하세요
cols = ['성분코드', '성분명', '영문명', 'CAS No', '구명칭']
ing_df = pd.DataFrame(columns=cols)

# HTML 테이블 -> 2차원 리스트로 파싱
def parse_table_to_list(table_tag):
    rows = []
    for row in table_tag.find_all('tr'):
        cells = row.find_all(['th', 'td'])
        cell_texts = [cell.get_text(strip=True) for cell in cells]
        if cell_texts:  # 빈 행 제거
            rows.append(cell_texts)
    return rows

# 페이지 루프
for num in range(1, 2164):
    url = f"https://kcia.or.kr/cid/search/ingd_list.php?page={num}"

    if num % 100 == 0:
        print(f"[현재 진행상황] : {num}")

    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('table', {"class": "bbs_list"})

        if table:
            data_2d = parse_table_to_list(table)
            if len(data_2d) > 1:
                data_rows = data_2d[1:]  # 첫 줄은 헤더이므로 제거
                tmpdf = pd.DataFrame(data_rows, columns=cols)
                ing_df = pd.concat([ing_df, tmpdf], ignore_index=True)
            else:
                print(f"[주의] {num} 페이지에 데이터 행이 없습니다.")
        else:
            print(f"[경고] {num} 페이지에 테이블이 없습니다.")

        time.sleep(0.3)  # 서버 부하 방지를 위한 딜레이

    except requests.RequestException as e:
        print(f"[에러] {num} 페이지 요청 실패: {e}")

ing_df.to_csv('ingredients_data.csv', index=False, encoding='utf-8-sig')


[현재 진행상황] : 100
[현재 진행상황] : 200
[현재 진행상황] : 300
[현재 진행상황] : 400
[현재 진행상황] : 500
[현재 진행상황] : 600
[현재 진행상황] : 700
[현재 진행상황] : 800
[현재 진행상황] : 900
[현재 진행상황] : 1000
[현재 진행상황] : 1100
[현재 진행상황] : 1200
[현재 진행상황] : 1300
[현재 진행상황] : 1400
[현재 진행상황] : 1500
[현재 진행상황] : 1600
[현재 진행상황] : 1700
[현재 진행상황] : 1800
[현재 진행상황] : 1900
[현재 진행상황] : 2000
[현재 진행상황] : 2100


In [17]:
ing_df = ing_df.astype({'성분코드':'int'})
ing_df = ing_df[['성분코드', '성분명', '영문명']].set_index('성분코드').sort_index()
ing_df.reset_index(inplace=True)
ing_df.head(10)

,성분코드,성분명,영문명
0,1,가공소금,
1,2,가지열매추출물,Solanum Melongena (Eggplant) Fruit Extract
2,3,구멍쇠미역추출물,Agarum Cribrosum Extract
3,4,루핀아미노산,Lupine Amino Acids
4,5,류신,Leucine
5,6,류코노스톡/무발효여과물,Leuconostoc/Radish Root Ferment Filtrate
6,7,백혈구추출물,Leukocyte Extract
7,8,리나칸투스 콤무니스추출물,Rhinacanthus Communis Extract
8,9,리날룰,Linalool
9,10,리날릴아세테이트,Linalyl Acetate


In [19]:
import re
pattern = r'\([^)]*\)'

for idx, row in ing_df.iterrows():
    tmp = ing_df.iloc[idx]['영문명']
    try:
        if '(' in tmp:
            txt = re.sub(pattern=pattern, repl='', string= tmp)
            txt = ' '.join(txt.split())
            ing_df.iloc[idx,2] = txt
    except:
        pass
        
ing_df['formatted_영문명'] = ing_df['영문명'].str.lower().str.replace(" ","-") 
# inci-decoder에 검색가능한 format으로 변경하여 컬럼 추가

ing_df.head(10) # 확인 한 번 해주고
ing_df.to_csv('ing_data.csv', index=False, encoding='utf-8-sig') # 저장

In [20]:
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
from urllib.request import Request, urlopen

In [21]:
ing_df = pd.read_csv('ing_data.csv',index_col=0)
ing_df['formatted_영문명'] = ing_df['formatted_영문명'].str.replace("/","-")

In [22]:
ing_list = list(ing_df['formatted_영문명'].dropna())

product_name = set() # 제품명
product_label = set() # 제품명 (formatted - 웹페이지 접근용)
search_failed = [] # 'formatted_영문명' 값으로 웹페이지 접근이 불가했던 건들 확인용도

In [25]:
def add_ing_products(tags):  # html tag 를 받아와 조회하여 
    for tag in tags:
        if tag.text not in product_name: # 중복된 데이터는 추가하지 않도록, tag의 제품명이 product_name 셋에 없는 경우에만 추가
            product_name.add(tag.text)
            product_label.add(tag.attrs['data-ga-eventlabel'][8:])

def next_page_exists(soup):
    if "Next" in soup.find(id="product").find_all("div")[-1].text: # Next라는 문자가 해당 태그안에 존재하는지 여부 확인
        return True
    else:
        return False

idx = 0

for ing in tqdm(ing_list):  # 성분 리스트의 각 성분(formatted)마다 
    url = 'https://incidecoder.com/ingredients/' + ing  # url 주소를 생성

    try:
        
        html = urlopen(url) 
        source = html.read()
        soup = BeautifulSoup(source, "html.parser")
        tags = soup.select("#product > div > a")  
        add_ing_products(tags)  

        if next_page_exists(soup): 
            nextpage = True

        while nextpage:
            nexturl = soup.find(id="product").find_all("a")[-1]['href']
            url = 'https://incidecoder.com' + nexturl 
            html = urlopen(url) 
            source = html.read()
            soup = BeautifulSoup(source, "html.parser")
            tags = soup.select("#product > div > a")
            add_ing_products(tags)

            if not next_page_exists(soup):
                nextpage = False

    except Exception:
        search_failed.append(ing)
        pass

    idx += 1

    # 100개마다 저장
    if idx % 100 == 0:
        product_all = pd.DataFrame(columns=['product_label'])
        product_all['product_label'] = list(product_label)
        product_all.to_csv(f'product_df_{idx}.csv', index=False)

# 마지막까지 저장 안 된 것 있으면 추가 저장
if idx % 100 != 0:
    product_all = pd.DataFrame(columns=['product_label'])
    product_all['product_label'] = list(product_label)
    product_all.to_csv(f'product_df_{idx}.csv', index=False)

100%|██████████| 20457/20457 [34:33:13<00:00,  6.08s/it]       


In [ ]:
import pandas as pd

# CSV 파일 읽기
df = pd.read_csv('product_df_20457.csv')

# 2행부터 (인덱스 1부터), 첫 번째 열의 값들을 리스트로 추출
product_lst = df.iloc[0:, 0].tolist()

# 결과 확인
print(product_lst)

['lissa-retinol-cream-with-spf30', 'vegamour-hydr-8-shampoo', 'dropology-vitamin-c-solution-15']


In [38]:
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd
import numpy as np

from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
import requests
import json

product_name = [] # 화장품 이름
ingredient_lst = [] # 화장품에 들어있는 성분을 리스트로 받음
formatted_ingredient_lst = [] # formatted 성분 표기명을 리스트로 받음
what_lst = [] # 성분이 어떤 효능이 있는지 리스트로 받음
failed_lst = [] # 크롤링 중 실패한 로그를 추적하기 위해
each_ingredient_lst = []

# 테이블을 2차원 리스트로 변환하는 함수 (parser_functions.make2d 대체)
def parse_table_to_list(table_tag):
    rows = []
    for row in table_tag.find_all('tr'):
        cells = row.find_all(['td', 'th'])
        cell_texts = [cell.get_text(strip=True) for cell in cells]
        if cell_texts:
            rows.append(cell_texts)
    return rows

cnt = 0

# 제품 목록 루프
for product in tqdm(product_lst, desc="제품 정보 수집 중"):
    url = f"https://incidecoder.com/products/{product}"

    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        table = soup.find('table', {"class": "product-skim fs16"})
        if not table:
            raise Exception("성분 테이블 없음")

        table_data = parse_table_to_list(table)
        if len(table_data) <= 1:
            raise Exception("성분 데이터 없음")

        # 첫 행은 헤더라서 제거
        df = table_data[1:]
        tmpdf = pd.DataFrame(df, columns=["Ingredient name", "what-it-does", "irr., com.", "ID-Rating"])

        # bold 처리된 성분 제거 (formatted_name이 없는 항목)
        bold_ings = [td.get_text(strip=True) for td in table.find_all('td', {'class': 'bold'})]
        tmpdf = tmpdf[~tmpdf['Ingredient name'].isin(bold_ings)].reset_index(drop=True)

        # formatted 성분명 추출
        formatted_ings = [
            tag['href'][13:] for tag in table.find_all('a', {'class': "black ingred-detail-link"})
        ]
        formatted_ingredient_lst.append(formatted_ings)

        # 데이터 수집
        ingredient_lst.append(tmpdf['Ingredient name'].tolist())
        what_lst.append(tmpdf["what-it-does"].tolist())
        product_name.append(product)

    except Exception as e:
        failed_lst.append([product, str(e)])


for lst in ingredient_lst:
    for ing in lst:
        each_ingredient_lst.append(ing)

each_formatted_ingredient_lst = []
for lst in formatted_ingredient_lst:
    for ing in lst:
        each_formatted_ingredient_lst.append(ing)

each_what_lst = []
for lst in what_lst:
    for does in lst:
        tmp = []
        for does in does.replace('\n','').replace('\u200b','').split(','): # 전처리
            tmp.append(does.strip())
        each_what_lst.append(tmp)
        

제품 정보 수집 중:   0%|          | 0/164098 [00:00<?, ?it/s]

In [39]:
# 1. 제품명과 성분 리스트로 DataFrame 만들기
prod_ing_df = pd.DataFrame({
    'product name': product_name,
    'ingredients': ingredient_lst
})

# 2. 성분 리스트를 문자열로 저장하기 위해 JSON 문자열로 변환
prod_ing_df['ingredients'] = prod_ing_df['ingredients'].apply(json.dumps)
prod_ing_df.to_csv('product_ingredients.csv', index=False)

# 1. 성분명과 효능 리스트로 DataFrame 생성
ingredient_df = pd.DataFrame({
    'ingredients': each_ingredient_lst,
    'what-it-does': each_what_lst
})

# 2. 중복 제거 (같은 성분명은 첫 번째 효능만 유지)
ingredient_df = ingredient_df.drop_duplicates(subset='ingredients')

# 3. 리스트를 문자열(JSON)로 변환해서 CSV 저장 가능하게 함
ingredient_df['what-it-does'] = ingredient_df['what-it-does'].apply(json.dumps)
ingredient_df.to_csv('ingredient_functions.csv', index=False)

In [61]:
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd
import numpy as np

from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
import requests

# CSV 파일 읽기
df = pd.read_csv('product_df_20457.csv')

# 2행부터 (인덱스 1부터), 첫 번째 열의 값들을 리스트로 추출
product_lst = df.iloc[0:, 0].tolist()
# 제품명 끝에 -2, -3, -4 와 같은 제품들만 한번 더 추출해서 그 위 아래값들을 조사

product_name = [] # 화장품 이름
ingredient_lst = [] # 화장품에 들어있는 성분을 리스트로 받음
formatted_ingredient_lst = [] # formatted 성분 표기명을 리스트로 받음
what_lst = [] # 성분이 어떤 효능이 있는지 리스트로 받음
failed_lst = [] # 크롤링 중 실패한 로그를 추적하기 위해
each_ingredient_lst = []

def parse_table_to_list(table_tag):
    rows = []
    for row in table_tag.find_all('tr'):
        cells = row.find_all(['td', 'th'])
        cell_texts = [cell.get_text(strip=True) for cell in cells]
        if cell_texts:
            rows.append(cell_texts)
    return rows

for product in tqdm(product_lst, desc="제품 정보 수집 중"):
    if product[-2] == "-" and product[-1].isdigit():
        num = int(product[-2::])
        for idx in range(1, 10):
            if idx == num:
                continue
            productURL = product
            if idx == 1:
                productURL = product[:-2]
            else:
                productURL = product[:-1] + str(idx)

            url = f"https://incidecoder.com/products/{productURL}"

            try:
                response = requests.get(url, timeout=10)
                response.raise_for_status()
                soup = BeautifulSoup(response.text, 'html.parser')

                table = soup.find('table', {"class": "product-skim fs16"})
                if not table:
                    raise Exception("성분 테이블 없음")

                table_data = parse_table_to_list(table)
                if len(table_data) <= 1:
                    raise Exception("성분 데이터 없음")

                # 첫 행은 헤더라서 제거
                df = table_data[1:]
                tmpdf = pd.DataFrame(df, columns=["Ingredient name", "what-it-does", "irr., com.", "ID-Rating"])

                # bold 처리된 성분 제거 (formatted_name이 없는 항목)
                bold_ings = [td.get_text(strip=True) for td in table.find_all('td', {'class': 'bold'})]
                tmpdf = tmpdf[~tmpdf['Ingredient name'].isin(bold_ings)].reset_index(drop=True)

                # formatted 성분명 추출
                formatted_ings = [
                    tag['href'][13:] for tag in table.find_all('a', {'class': "black ingred-detail-link"})
                ]
                formatted_ingredient_lst.append(formatted_ings)

                # 데이터 수집
                ingredient_lst.append(tmpdf['Ingredient name'].tolist())
                what_lst.append(tmpdf["what-it-does"].tolist())
                product_name.append(productURL)

            except Exception as e:
                break

for lst in ingredient_lst:
    for ing in lst:
        each_ingredient_lst.append(ing)

each_formatted_ingredient_lst = []
for lst in formatted_ingredient_lst:
    for ing in lst:
        each_formatted_ingredient_lst.append(ing)

each_what_lst = []
for lst in what_lst:
    for does in lst:
        tmp = []
        for does in does.replace('\n','').replace('\u200b','').split(','): # 전처리
            tmp.append(does.strip())
        each_what_lst.append(tmp)

# 1. 제품명과 성분 리스트로 DataFrame 만들기
prod_ing_df = pd.DataFrame({
    'product name': product_name,
    'ingredients': ingredient_lst
})

# 2. 성분 리스트를 문자열로 저장하기 위해 JSON 문자열로 변환
prod_ing_df['ingredients'] = prod_ing_df['ingredients'].apply(json.dumps)
prod_ing_df.to_csv('product_ingredients_append_edge_case.csv', index=False)

# 1. 성분명과 효능 리스트로 DataFrame 생성
ingredient_df = pd.DataFrame({
    'ingredients': each_ingredient_lst,
    'what-it-does': each_what_lst
})

# 2. 중복 제거 (같은 성분명은 첫 번째 효능만 유지)
ingredient_df = ingredient_df.drop_duplicates(subset='ingredients')

# 3. 리스트를 문자열(JSON)로 변환해서 CSV 저장 가능하게 함
ingredient_df['what-it-does'] = ingredient_df['what-it-does'].apply(json.dumps)
ingredient_df.to_csv('ingredient_functions_append_edge_case.csv', index=False)
        

제품 정보 수집 중:   0%|          | 0/164098 [00:00<?, ?it/s]

In [65]:
# CSV 파일 읽기
df1 = pd.read_csv('ingredient_functions.csv')
df2 = pd.read_csv('ingredient_functions_append_edge_case.csv')

# 1열 기준 병합 (첫 번째 열의 이름을 알아야 합니다. 모를 경우 df1.columns[0]으로 가져옵니다)
key_col = df1.columns[0]

# 두 데이터프레임을 합치고, 중복 제거
merged_df = pd.concat([df1, df2]).drop_duplicates(subset=[key_col])

# 결과 저장
merged_df.to_csv('ingredient_functions_merged.csv', index=False)
print("✅ 병합 완료! 결과는 ingredient_functions_merged.csv로 저장되었습니다.")

✅ 병합 완료! 결과는 ingredient_functions_merged.csv로 저장되었습니다.


In [66]:
# CSV 파일 읽기
df1 = pd.read_csv('product_ingredients.csv')
df2 = pd.read_csv('product_ingredients_append_edge_case.csv')

# 1열 기준 병합 (첫 번째 열의 이름을 알아야 합니다. 모를 경우 df1.columns[0]으로 가져옵니다)
key_col = df1.columns[0]

# 두 데이터프레임을 합치고, 중복 제거
merged_df = pd.concat([df1, df2]).drop_duplicates(subset=[key_col])

# 결과 저장
merged_df.to_csv('product_ingredients_merged.csv', index=False)
print("✅ 병합 완료! 결과는 product_ingredients_merged.csv로 저장되었습니다.")

✅ 병합 완료! 결과는 product_ingredients_merged.csv로 저장되었습니다.
